In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd

title = []
date = []
desc = []
review = []
company = []
i = 1
count = 0

url_head = 'https://www.trustpilot.com/review/www.easports.com'
page = requests.get(url_head)
soup = BS(page.content, 'html.parser')
h = soup.find('section', class_ = 'styles_businessInformation__6ks_E')
c=h.find('span', class_ = 'typography_display-s__qOjh6 typography_appearance-default__AAY17 title_displayName__TtDDM').get_text(strip=True)
r = h.find('span', class_ = 'typography_body-l__KUYFJ typography_appearance-subtle__8_H2l styles_text__W4hWi').get_text(strip=True)
total_reviews = r[:r.find('•')] #not using it anywhere

while count < 500:
    url = 'https://www.trustpilot.com/review/www.easports.com?page={}'.format(i)
    i = i + 1
    page = requests.get(url)
    soup = BS(page.content, 'html.parser')
    lists = soup.find_all('section', class_ = 'styles_reviewContentwrapper__zH_9M')
    for list in lists:
        count = count + 1
        company.append(c)
        title.append(list.find('h2', class_ = 'typography_heading-s__f7029 typography_appearance-default__AAY17').text)
        review.append(list.find('div', class_ = 'styles_reviewHeader__iU9Px')['data-service-review-rating'])
        d = list.find('p', class_ = 'typography_body-m__xgxZ_ typography_appearance-default__AAY17 typography_color-black__5LYEn').text
        date.append(d[20:])
        p = list.find('p', class_ = 'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn')
        if p is None:
            desc.append('')
        else:
            desc.append(p.text)

dic = {'companyName': company, 'datePublished': date, 'ratingValue': review, 'reviewTitle': title, 'reviewBody':desc}  
df = pd.DataFrame(dic)
df.to_csv('scaper.csv', index=False)